In [2]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os
import importlib


module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='preprocessing/hosp_module_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)

root_dir = "/data/corpora_alpha/MIMIC/MIMIC_IV_2.2/files"

import day_intervals_cohort
from day_intervals_cohort import *

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import data_generation
import evaluation

import feature_selection_hosp
from feature_selection_hosp import *

# import train
# from train import *


import ml_models
from ml_models import *

import dl_train
from dl_train import *

import tokenization
from tokenization import *


import behrt_train
from behrt_train import *

import feature_selection_icu
from feature_selection_icu import *
import fairness
import callibrate_output

In [3]:
importlib.reload(day_intervals_cohort)
import day_intervals_cohort
from day_intervals_cohort import *

importlib.reload(day_intervals_cohort_v2)
import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

importlib.reload(data_generation_icu)
import data_generation_icu
importlib.reload(data_generation)
import data_generation

importlib.reload(feature_selection_hosp)
import feature_selection_hosp
from feature_selection_hosp import *

importlib.reload(feature_selection_icu)
import feature_selection_icu
from feature_selection_icu import *

importlib.reload(tokenization)
import tokenization
from tokenization import *

importlib.reload(ml_models)
import ml_models
from ml_models import *

importlib.reload(dl_train)
import dl_train
from dl_train import *

importlib.reload(behrt_train)
import behrt_train
from behrt_train import *

importlib.reload(fairness)
import fairness

importlib.reload(callibrate_output)
import callibrate_output

importlib.reload(evaluation)
import evaluation

In [4]:
def eda_basic(dataframes, dataframes_names = None, log_path=None):
    '''
    Perform basic exploratory data analysis on a list of pandas DataFrames.
    Args:
    dataframes (list): A list of pandas DataFrames.
    dataframes_names (list, optional): A list of names for the DataFrames. If provided, the names will be used to distinguish between the DataFrames in the results.
    log_path (str, optional): The path to save the log file. If provided, the results will be logged to the file.
    '''
    if(len(dataframes) == 0):
        print("No dataframes provided.")
        return
    
    if dataframes_names:
        if len(dataframes) != len(dataframes_names):
            print("The number of dataframes and the number of names provided do not match.")
            return

    if log_path:
        log_file_path = os.path.join(log_path, 'eda_basic_log.txt')
        os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
        log_file = open(log_file_path, 'a')
        log_file.write(f"{'='*40}\n")
        log_file.write(f"{pd.Timestamp.now()}\n")
        log_file.write(f"{'='*40}\n")
        
    for df_index in range(len(dataframes)):
        if dataframes_names:
            print(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}")
            if log_path:
                log_file.write(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}\n")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
            results = [
                ("First few rows of the dataframe:\n", dataframes[df_index].head()),
                ("Columns in the dataframe:\n", dataframes[df_index].columns),
                ("Number of rows in the dataframe:\n", dataframes[df_index].shape[0]),
                ("Missing values in the dataframe:\n", dataframes[df_index].isnull().sum()),
                ("Number of duplicate rows in the dataframe:\n", dataframes[df_index].duplicated().sum()),
                ("Data types of columns:\n", dataframes[df_index].dtypes),
                ("Number of unique values in each column:\n", dataframes[df_index].nunique()),
            ]
            for label, result in results:
                print(label)
                print(result)
                print()
                if log_path:
                    log_file.write(f"{label}{result}\n\n")
            if log_path:
                log_file.write(f"{'X'*40}\n")
    if log_path:
        log_file.close()

In [77]:
#Uncomment if need to generate the cohort again
imputation_cohort_var_non_ICU = day_intervals_cohort_v2.extract_imputation_data("Non-ICU", root_dir)
#imputation_cohort_var_ICU = day_intervals_cohort_v2.extract_imputation_data("ICU", root_dir)

EXTRACTING FOR: | NON-ICU | IMPUTATION |


In [5]:
imputation_cohort_var_non_ICU = "cohort_non-icu_imputation"
imputation_cohort_var_ICU = "cohort_icu_imputation"
imputation_cohort_dataset_non_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var_non_ICU}.csv.gz")
imputation_cohort_dataset_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var_ICU}.csv.gz")

In [50]:
eda_basic([imputation_cohort_dataset_non_ICU, imputation_cohort_dataset_ICU], ['Non-ICU', 'ICU'], './data/summary/')

****************************************
Non-ICU
****************************************
First few rows of the dataframe:

   subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P874LG  TRANSFER FROM HOSPITAL   
1        EW EMER.            P09Q6Y          EMERGENCY ROOM   
2        EW EMER.            P60CC5          EMERGENCY ROOM   
3        EW EMER.            P30KEH          EMERGENCY ROOM   
4  EU OBSERVATION            P51VDL          EMERGENCY ROOM   

  discharge_location i

In [8]:
diag_flag = False
lab_flag = True
proc_flag = False
med_flag = False
cohort_output = imputation_cohort_var_non_ICU
version_path = "mimiciv/2.2"
feature_nonicu_imputation(cohort_output,version_path,diag_flag,lab_flag,proc_flag,med_flag)

[EXTRACTING LABS DATA]


In [28]:
diag_pd = pd.read_csv(f"./data/features/imputation_non_icu_preproc_diag.csv.gz")
#proc_pd = pd.read_csv(f"./data/features/imputation_non_icu_preproc_proc.csv.gz")
eda_basic([diag_pd, proc_pd], ['diagnosis', 'procedures'], './data/summary/')

****************************************
diagnosis
****************************************
First few rows of the dataframe:

   subject_id   hadm_id  seq_num icd_code  icd_version root_icd10_convert root
0    10000032  22595853        1     5723            9               K750  K75
1    10000032  22595853        2    78959            9               R109  R10
2    10000032  22595853        3     5715            9               K700  K70
3    10000032  22595853        4    07070            9               B150  B15
4    10000032  22595853        5      496            9               J449  J44

Columns in the dataframe:

Index(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version',
       'root_icd10_convert', 'root'],
      dtype='object')

Number of rows in the dataframe:

4756326

Missing values in the dataframe:

subject_id                 0
hadm_id                    0
seq_num                    0
icd_code                   0
icd_version                0
root_icd10_convert 

In [94]:
med_pd = pd.read_csv(f"./data/features/imputation_non_icu_preproc_med.csv.gz")
eda_basic([med_pd], ['medications'], './data/summary/')

****************************************
medications
****************************************
First few rows of the dataframe:

   subject_id   hadm_id  pharmacy_id       poe_id  poe_seq order_provider_id  \
0    10000032  22595853     11700683  10000032-34     34.0            P76JEQ   
1    10000032  22595853     14779570  10000032-22     22.0            P76JEQ   
2    10000032  22595853     19796602  10000032-50     50.0            P260SK   
3    10000032  22595853     20256254  10000032-32     32.0            P76JEQ   
4    10000032  22595853     28781051  10000032-27     27.0            P76JEQ   

             starttime             stoptime drug_type  \
0  2180-05-07 01:00:00  2180-05-07 22:00:00      MAIN   
1  2180-05-07 00:00:00  2180-05-07 22:00:00      MAIN   
2  2180-05-08 08:00:00  2180-05-07 22:00:00      MAIN   
3  2180-05-07 01:00:00  2180-05-07 22:00:00      MAIN   
4  2180-05-07 00:00:00  2180-05-07 22:00:00      MAIN   

                          drug formulary_drug_cd

In [6]:
labs_pd = pd.read_csv(f"./data/features/imputation_non_icu_preproc_labs.csv.gz")
eda_basic([labs_pd], ['labs'], './data/summary/')

****************************************
labs
****************************************
First few rows of the dataframe:

   labevent_id  subject_id     hadm_id  specimen_id  itemid order_provider_id  \
0          111    10000032  22595853.0     39504011   51464               NaN   
1          300    10000032  22841357.0     37310858   51464               NaN   
2         1819    10000248  20600184.0     15125842   51464               NaN   
3         2538    10000764  27897940.0     94338985   51464               NaN   
4         2813    10000826  20032235.0     93627294   51464               NaN   

             charttime            storetime value  valuenum valueuom  \
0  2180-05-07 00:10:00  2180-05-07 00:40:00   NaN       NaN    mg/dL   
1  2180-06-26 22:45:00  2180-06-27 00:38:00   NaN       NaN    mg/dL   
2  2192-11-29 19:50:00  2192-11-29 20:14:00   NaN       NaN    mg/dL   
3  2132-10-15 11:25:00  2132-10-15 11:52:00   NaN       NaN    mg/dL   
4  2146-12-08 22:22:00  2146-12-

In [8]:
labs_pd.label.value_counts()

label
Glucose                    2207606
Potassium                  1945621
Sodium                     1926511
Hematocrit                 1912876
Chloride                   1911014
                            ...   
Chloride, Pleural                1
CD56 %                           1
CD56 Absolute Count              1
Uric Acid, Joint Fluid           1
Creatinine, Joint Fluid          1
Name: count, Length: 680, dtype: int64

In [10]:
#print the names of all labels in ascending order
labs_pd.valueuom.value_counts().index.tolist()

['mg/dL',
 'mEq/L',
 '%',
 'K/uL',
 'g/dL',
 'fL',
 'IU/L',
 'sec',
 'm/uL',
 'pg',
 'mm Hg',
 'units',
 'mmol/L',
 'ng/mL',
 '#/hpf',
 ' ',
 'ug/mL',
 'ug/dL',
 '/hpf',
 '#/uL',
 'mOsm/kg',
 'pg/mL',
 '#/lpf',
 'uIU/mL',
 'mg/L',
 'Ratio',
 'ng/dL',
 'mm/hr',
 'umol/L',
 'L/min',
 'mg/mg',
 'IU/mL',
 '+/-',
 'log10 IU/mL',
 'U/mL',
 'mIU/mL',
 'mg/24hr',
 'mL',
 'ng/mL FEU',
 'mg/g',
 'GPL',
 'MPL',
 'U/g/Hb',
 'log10 copies/mL',
 'U',
 'mL/min',
 'hrs',
 'log10 cop/mL',
 'U/L',
 'nmol/L',
 'Pos/Neg',
 'Score',
 'min']

In [11]:
# Load omr table from the hosp 
omr_pd = pd.read_csv(f"/data/corpora_alpha/MIMIC/MIMIC_IV_2.2/files/mimiciv/2.2/hosp/omr.csv.gz")
omr_pd.head()


,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15


In [30]:
# Add the h_adm_id to the omr dataframe
# This needs to be done such that the chartdate from omr is between the admittime and dischtime from admissions table
# This is done to ensure that the omr data is within the admission time of the patient

import pandas as pd

# Load admissions table
admissions_pd = pd.read_csv(f"/data/corpora_alpha/MIMIC/MIMIC_IV_2.2/files/mimiciv/2.2/hosp/admissions.csv.gz")

# Convert chartdate, admittime, and dischtime to datetime
omr_pd['chartdate'] = pd.to_datetime(omr_pd['chartdate'])
admissions_pd['admittime'] = pd.to_datetime(admissions_pd['admittime'])
admissions_pd['dischtime'] = pd.to_datetime(admissions_pd['dischtime'])

# Merge omr and admissions dataframes on subject_id
merged_df = pd.merge(omr_pd, admissions_pd[['subject_id', 'hadm_id', 'admittime', 'dischtime']], on='subject_id', how='left')

# Filter rows where chartdate is between admittime and dischtime
filtered_df = merged_df[(merged_df['chartdate'] >= merged_df['admittime']) & (merged_df['chartdate'] <= merged_df['dischtime'])]

# Drop unnecessary columns
filtered_df = filtered_df.drop(columns=['admittime', 'dischtime'])

# Save the result back to omr_pd
omr_pd = filtered_df


In [39]:
# Now we want to use the following result_names from the omr_pd and add them as columns to the admissions dataframe
# BMI (kg/m2)	
# Weight (Lbs)	
# Blood Pressure	
# Height (Inches)	

# Pivot the omr_pd dataframe to have result_names as columns
pivot_df = omr_pd.pivot_table(index=['subject_id', 'hadm_id', 'chartdate'], columns='result_name', values='result_value', aggfunc='first').reset_index()

In [40]:
#can you tell me the missing values in the columns
eda_basic([admissions_pd], ['admissions'], './data/summary/')

****************************************
admissions
****************************************
First few rows of the dataframe:

   subject_id   hadm_id           admittime           dischtime deathtime  \
0    10000032  22595853 2180-05-06 22:23:00 2180-05-07 17:15:00       NaN   
1    10000032  22841357 2180-06-26 18:27:00 2180-06-27 18:49:00       NaN   
2    10000032  25742920 2180-08-05 23:44:00 2180-08-07 17:50:00       NaN   
3    10000032  29079034 2180-07-23 12:35:00 2180-07-25 17:55:00       NaN   
4    10000068  25022803 2160-03-03 23:16:00 2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P874LG  TRANSFER FROM HOSPITAL   
1        EW EMER.            P09Q6Y          EMERGENCY ROOM   
2        EW EMER.            P60CC5          EMERGENCY ROOM   
3        EW EMER.            P30KEH          EMERGENCY ROOM   
4  EU OBSERVATION            P51VDL          EMERGENCY ROOM   

  discharge_location insurance 